In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    
    def inc(self, numOccur):
        self.count += numOccur
    
    def disp(self, ind=1):
        print('  ' * ind + f'{self.name} {self.count}')  
        for child in self.children.values():
            child.disp(ind + 1)

In [2]:
rootNode = treeNode('pyramid', 9, None)

In [3]:
rootNode.children['eye']=treeNode('eye', 13, None)

In [4]:
rootNode.disp()

  pyramid 9
    eye 13


In [5]:
rootNode.children['phoenix']=treeNode('eyephoenix', 3, None)

In [6]:
rootNode.disp()

  pyramid 9
    eye 13
    eyephoenix 3


In [7]:
def updateHedder(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [8]:
def updateTree(items, inTree, headderTable, count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headderTable[items[0]][1] == None:
            headderTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHedder(headderTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headderTable, count)

In [9]:
def createTree(dataSet, minSup=1):
    hedderTable = {}
    for trans in dataSet:
        for item in trans:
            hedderTable[item] = hedderTable.get(item, 0) + dataSet[trans]
    # for k in hedderTable.keys():
    #     if hedderTable[k] < minSup:
    #         del(hedderTable[k])
    keys_to_remove = [k for k in hedderTable.keys() if hedderTable[k] < minSup]
    for k in keys_to_remove:
        del hedderTable[k]
    freqItemSet = set(hedderTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    for k in hedderTable:
        hedderTable[k] = [hedderTable[k], None]
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = hedderTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key= lambda p:p[1], reverse=True)]
            updateTree(orderedItems, retTree, hedderTable, count)
    return retTree, hedderTable
# def createTree(dataSet, minSup=1):
#     hedderTable = {}
#     # Count item frequencies
#     for trans in dataSet:
#         for item in trans:
#             hedderTable[item] = hedderTable.get(item, 0) + dataSet[trans]
    
#     # Remove items not meeting minSup
#     keys_to_remove = [k for k in hedderTable.keys() if hedderTable[k] < minSup]
#     for k in keys_to_remove:
#         del hedderTable[k]

#     freqItemSet = set(hedderTable.keys())
#     if len(freqItemSet) == 0:
#         return None, None

#     # Initialize header table with [count, None]
#     for k in hedderTable:
#         hedderTable[k] = [hedderTable[k], None]
    
#     # Build FP-Tree
#     retTree = treeNode('Null Set', 1, None)
#     for tranSet, count in dataSet.items():
#         localD = {}
#         for item in tranSet:
#             if item in freqItemSet:
#                 localD[item] = hedderTable[item][0]
#         if len(localD) > 0:
#             orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
#             updateTree(orderedItems, retTree, hedderTable, count)
    
#     return retTree, hedderTable


In [10]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [11]:
def createInItSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [12]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [13]:
inItSet = createInItSet(simpDat)
# inItSet = createInitSet(simpDat)
inItSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [14]:
myFPTree, myHedderTab = createTree(inItSet, 3)
myFPTree.disp()

  Null Set 1
    z 5
      r 1
      x 3
        y 3
          s 2
            t 2
          r 1
            t 1
    x 1
      s 1
        r 1


In [15]:
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [16]:
def findPrefixPath(basePat, treeNode):
    condPats = {}
    while treeNode:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [17]:
findPrefixPath('x', myHedderTab['x'][1])

{frozenset({'z'}): 3}

In [18]:
findPrefixPath('z', myHedderTab['z'][1])

{}

In [19]:
findPrefixPath('r', myHedderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'x', 'y', 'z'}): 1}

In [36]:
def mineTree(inTree, headerTable, minSup, prefix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key= lambda p: p[1][0])]
    for basePat in bigL:
        newFreqSet = prefix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPatBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPatBases, minSup)
        if myHead != None:
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)
            print(f'conditional tree for:{newFreqSet}')
            myCondTree.disp(1)
    # return myCondTree

In [37]:
freqItems = []

In [38]:
mineTree(myFPTree, myHedderTab, 3, set([]), freqItems)

conditional tree for:{'x', 'y'}
  Null Set 1
    z 3
conditional tree for:{'y'}
  Null Set 1
    z 3
      x 3
conditional tree for:{'s'}
  Null Set 1
    x 3
conditional tree for:{'t'}
  Null Set 1
    z 2
      x 2
        y 2
    x 1
      z 1
        y 1
conditional tree for:{'x'}
  Null Set 1
    z 3


In [39]:
freqItems

[{'r'},
 {'y'},
 {'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'z'},
 {'t', 'x'},
 {'t', 'y'},
 {'x'},
 {'x', 'z'},
 {'z'}]

In [40]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]

In [41]:
initSet = createInItSet(parsedDat)

In [42]:
myFPTree, myHedderTab = createTree(initSet, 100000)

In [43]:
myFreqList = []
mineTree(myFPTree, myHedderTab, 100000, set([]), myFreqList)

conditional tree for:{'1'}
  Null Set 1
    6 107404
conditional tree for:{'11', '3'}
  Null Set 1
    6 117401
conditional tree for:{'3'}
  Null Set 1
    6 186289
      11 117401
    11 9718
conditional tree for:{'11'}
  Null Set 1
    6 261773


In [44]:
len(myFreqList)

9

In [45]:
myFreqList

[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]